In [2]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
import random
import io
import string
from sklearn.metrics.pairwise import cosine_similarity
import MeCab 
import numpy as np

In [101]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

In [102]:
mecab = MeCab.Tagger('-Owakati')

#p = Path("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/Train_Test_Data/japanese/corpus_ver14.csv", 
                 header=0, usecols=["page", "text"])

In [5]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/assets/learning/stop_word.txt")
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_w = fr.read().splitlines()

len(stop_w)

newlist = ['ます', '端末', 'する', '設定', '表示', 'mobicontrol', '選択', '入力', 'アプリ', 'ユーザ', '構成', 'でき', 
           '画面', '接続', 'ルール', 'ない', 'インストール', 'です', '指定', '管理', 'ad', 'ください', 'この', 'クリック', 
           '参照', 'ボタン', 'あり', 'チェック', 'コンソール', 'または', '証明', 'id', 'android', '項目', 'なり', 'アクセス',
           'データ', '対象', '適用', 'いる', 'pc', 'その']
stop_words_ja = stop_w + newlist

In [135]:
len(stop_words_ja)

372

In [107]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [108]:
classes = df.page.unique()
c_size = 100

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    data = ''.join(list(arr_index))
    
    all_data = clean_text(data)
    
    arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    num = arr_words.shape[0]//c_size
    full = c_size * num
    rest = arr_words.shape[0] - full
    
    pad = np.zeros([c_size-rest], dtype=int)
    sc = np.concatenate((arr_words,pad))
    features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(features)
    df_v["cls"] = i
    
    result = pd.concat([result, df_v]) 

    
#print(result)
#result.to_csv('processed.csv')

In [109]:
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
#tog.to_csv("tog.csv")
sent = list(tog.values)

In [ ]:
len(sent)
stop_words_ja

In [164]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words_ja, max_df=0.95)
vectors = vectorizer.fit_transform(sent)

In [165]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ',
                            'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか',
                            'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら',
                            'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [166]:
# idf score 
#idf = vectorizer.idf_
# visulalize idf score
#print(dict(zip(vectorizer.get_feature_names(), idf)))

weights = np.asarray(vectors.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})

In [167]:
TermsWeight = weights_df.sort_values(by='weight', ascending=False).head(50)

In [168]:
TermsWeight.term

4899         登録
2403        サーバ
2316       グループ
2971     プロファイル
2824       ファイル
5313         認証
3449         作成
2775      パスワード
4828         現れ
1617    windows
2204     エージェント
3067       メニュー
2587     ダウンロード
1951         ませ
4317         押し
4249         情報
1589        vpn
4437         文字
2016      アカウント
2051       アドレス
4319         押す
2003         れる
3804         可能
1871        として
3831         同期
3670       割り当て
2573        タップ
1860        できる
1689         ある
4405         操作
4226         必要
2852       フォルダ
3661         削除
3810         右端
429          ds
84        apple
4641         機能
4911         監視
1549        url
595      google
3651         利用
5317         説明
2452      スクリプト
3081        メール
3967         変更
5435         追加
864      manual
4184         当該
3245         一覧
4056         実行
Name: term, dtype: object

In [170]:
# Read a question from the user
question = [input('Please enter a question: \n')]
question = vectorizer.transform(question)

# Rank all the questions using cosine similarity to the input question
rank = cosine_similarity(question, vectorizer.transform(sent))

# Grab the top 5
top = np.argsort(rank, axis=-1).T[-10:]
#print(top)

for item in range(len(top)):
    idx = top[item,0]
    
    # if the dataset file structure "Quesition | Answer" then use following line  
    #print(data['Answer'].iloc[item].values[0])
    print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
    print("\n ########## \n")

Please enter a question: 
AndroidEnterise
PageID:  135 idx 2639 = 端末 画像 を 印刷 端末 画像 の 変化 を ビデオ 録画 ビデオ 録画 の 中止 ビデオ 録画 の 終了 管理 者 権限 モード に 変える Windows Embedded 端末 で は 次 の 詳細 設定 で 端末 の 機能 に 制限 を 加え ます が これ は 管理 者 権限 モード で は 制限 から 開放 さ れ ます ブラックリストアプリ の 起動 停止 カメラ など 入出力 に関する 機能 の 停止 ランチャー の 強制 起動 電話 の 発 着信 制限 随意 の 電話 番号 に 電話 かけ させ ない 受呼 さ せ ない 管理 者 モード に 変える 方法 は 次 の

 ########## 

PageID:  135 idx 2638 = 稼働 中 の プロセス の 一覧 が 表示 さ れ ます 端末 リソース 右 下 の ペイン に は 端末 の リソース が 表示 さ れ ます 図 コマンド として ? を 入力 し コマンド の 種類 全て を 表示 し た 画面 ツール バー 図 の 左上 に ツール バー が 表示 さ れ て い ます 端末 画像 の 縮小 端末 画像 の 拡大 端末 画像 の 回転 端末 画像 の 色 品質 通信 回線 が 遅い 時 は 低 品質 へ 端末 画像 を キャプチャ 画像 を コンソール コンピュータ に 保存 端末 画像 を クリップ ボード へ

 ########## 

PageID:  135 idx 2637 = の 端末 画像 の 右 の 「 シェル 」 タブ 赤 矢印 を 選択 する と 端末 画像 の 部分 が 図 の よう に コマンド 入力 画面 に 変わり ます Windows Embedded の コマンド や MobiControl スクリプト を 使っ て コマンド を 入力 し 端末 を 操作 し ます MobiControl スクリプト は 「 」 を 参照 ください アプリケーションマネージャーツール 右 上 の ペイン の 「 アプリケーションマネージャーツール 」 ボタン を 押す と 端末 に 搭載 し て いる アプリケーショ